In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import math
import keras.backend as K
import csv
from tqdm import tqdm
import re
from PIL import Image
import PIL


### Frequency


In [ ]:
words_frq = []
frq = []
word_frq = []
sum_frq = 0
with open("Datasets/meaningful_words_with_frq.csv", 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        if len(row[0]) > 20: 
            continue
        words_frq.append([row[0], int(row[1])])
        frq.append(int(row[1]))
        sum_frq += int(row[1])


In [ ]:
words_frq.sort(key = lambda x: x[1], reverse=True)


In [ ]:
hfw = words_frq[:int(len(words_frq)/2)]
lfw = words_frq[int(len(words_frq)/2):]
highf_w = [item[0] for item in hfw]
lowf_w = [item[0] for item in lfw]
hf = frq[:int(len(words_frq)/2)]
lf = frq[int(len(words_frq)/2):]
ref = lf[0]
print(ref)

## Multiply images by Frq

In [ ]:
entries = os.listdir('ImageDataset/TrainSet/word/')
trainwords = []
trainwords_path = []
for e in entries:
    trainwords.append(e.split('_')[1].split('.')[0])
    trainwords_path.append('ImageDataset/TrainSet/word/'+e)
    
entries = os.listdir('ImageDataset/TestSet/word/')
testwords = []
testwords_path = []
for e in entries:
    testwords.append(e.split('_')[1].split('.')[0])
    testwords_path.append('ImageDataset/TestSet/word/'+e)

In [ ]:
num_extras = []
hf_count = 0
for w in tqdm(trainwords):
    if w in highf_w:
        hf_count += 1
        index = highf_w.index(w)
        f = hf[index]
        num_extras.append(int((int(f/ref)-1)/70))
        
print(f"# hf: {hf_count}  # lf: {len(trainwords)-hf_count}")

In [ ]:
total = 0
for i in num_extras:
    total+=i
print(f"Total number of extra images {total}")

In [ ]:
# train set
pbar = tqdm(total = total)
for w, p, n in tqdm(zip(trainwords, trainwords_path, num_extras)):
    im1 = Image.open(p)
    im2 = im1.copy()
    new_path = p.split('.')[0]
    for i in range(n):
            im2.save(new_path+"."+str(i)+'.png', "PNG")
    pbar.update(n)

# CNN

In [3]:
train = ImageDataGenerator(rescale=1/255)
test = ImageDataGenerator(rescale=1/255)
batch_size = 32
train_dataset = train.flow_from_directory("ImageDataset/TrainSet/",
                                          target_size=(30,150),
                                          batch_size = batch_size,
                                          class_mode = 'categorical')
                                         
test_dataset = test.flow_from_directory("ImageDataset/TestSet/",
                                          target_size=(30,150),
                                          batch_size = batch_size,
                                          class_mode = 'categorical')

Found 11876 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


In [4]:
test_dataset.class_indices


{'nonword': 0, 'word': 1}

### Network Structure 1

In [5]:
model = tf.keras.models.Sequential()

model.add(keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(30,150,3)))
model.add(keras.layers.MaxPool2D(2,2))
# model.add(keras.layers.BatchNormalization())


model.add(keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))
# model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.2))

model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))
# model.add(keras.layers.BatchNormalization())


model.add(keras.layers.Flatten())


model.add(keras.layers.Dense(256,activation='relu'))
# model.add(keras.layers.BatchNormalization())


# model.add(keras.layers.Dense(1,activation='sigmoid'))
model.add(keras.layers.Dense(2, activation='softmax'))


2021-12-24 14:22:39.383874: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-24 14:22:39.383900: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-24 14:22:39.383922: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mahvash-pc): /proc/driver/nvidia/version does not exist
2021-12-24 14:22:39.384144: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Compile

In [6]:
model.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 148, 32)       896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 12, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 36, 64)        0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 6, 36, 64)         0         
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 34, 128)        7

In [7]:

steps_per_epoch = len(train_dataset)

model.fit(train_dataset,
         steps_per_epoch = steps_per_epoch,
         epochs = 10,
         validation_data = test_dataset
       
         )

Epoch 1/10
372/372 [==============================] - 29s 76ms/step - loss: 0.5551 - accuracy: 0.6847 - val_loss: 0.4927 - val_accuracy: 0.7517
Epoch 2/10
372/372 [==============================] - 31s 84ms/step - loss: 0.3885 - accuracy: 0.8100 - val_loss: 0.3959 - val_accuracy: 0.8095
Epoch 3/10
372/372 [==============================] - 28s 76ms/step - loss: 0.3249 - accuracy: 0.8498 - val_loss: 0.3420 - val_accuracy: 0.8425
Epoch 4/10
372/372 [==============================] - 27s 72ms/step - loss: 0.2754 - accuracy: 0.8769 - val_loss: 0.3507 - val_accuracy: 0.8375
Epoch 5/10
372/372 [==============================] - 26s 71ms/step - loss: 0.2389 - accuracy: 0.8912 - val_loss: 0.3085 - val_accuracy: 0.8643
Epoch 6/10
372/372 [==============================] - 27s 72ms/step - loss: 0.2209 - accuracy: 0.9062 - val_loss: 0.3048 - val_accuracy: 0.8585
Epoch 7/10
372/372 [==============================] - 28s 76ms/step - loss: 0.1935 - accuracy: 0.9138 - val_loss: 0.3022 - val_accuracy:

In [14]:
import pickle
outfile = open("check_points/CNN_model",'wb')
pickle.dump(model,outfile)
outfile.close()

2021-12-24 14:39:30.042276: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ram://fc937906-24e6-4cba-bd2e-25fd4912e5a3/assets


## Train Set Results

In [23]:


entries = os.listdir('ImageDataset/TrainSet/nonword/')
nonwords = []
for e in entries:
    nonwords.append(e)


    
entries = os.listdir('ImageDataset/TrainSet/word/')
words = []
for e in entries:
    words.append(e)

In [24]:
prob = []
all_set = []
for e in tqdm(nonwords):
    word = (e.split('_'))[1].split('.')[0]
    if word in all_set:
        continue
    all_set.append(word)
    filename = "ImageDataset/TrainSet/nonword/" + e
    img = image.load_img(filename,target_size=(30,150))
 
    Y = image.img_to_array(img)
    X = np.expand_dims(Y,axis=0)
    X = X/255
    val = model.predict(X)
    prob.append([word, round(float(val[0][0]), 6), round(float(val[0][1]), 6),0])
    


100%|███████████████████████████████████████| 5000/5000 [03:12<00:00, 26.00it/s]


In [25]:
for e in tqdm(words):
    word = (e.split('_'))[1].split('.')[0]
    if word in all_set:
        continue
    all_set.append(word)
    filename = "ImageDataset/TrainSet/word/" + e
    img = image.load_img(filename,target_size=(30,150))
 
    Y = image.img_to_array(img)
    X = np.expand_dims(Y,axis=0)
    X = X/255
    val = model.predict(X)
    prob.append([word, round(float(val[0][0]), 6), round(float(val[0][1]), 6),1])



100%|███████████████████████████████████████| 6876/6876 [03:13<00:00, 35.54it/s]


In [26]:
with open("Datasets/train_dataset_prob.csv", 'w', newline='') as file:
    writer = csv.writer(file)
    for k in tqdm(prob):
        writer.writerow(k)

100%|█████████████████████████████████| 10000/10000 [00:00<00:00, 336721.50it/s]


In [27]:
with open("Datasets/train_dataset_unique_words.csv", 'w', newline='') as file:
    writer = csv.writer(file)
    for k in tqdm(prob):
        writer.writerow([k[0], k[3]])

100%|█████████████████████████████████| 10000/10000 [00:00<00:00, 485075.69it/s]
